In [15]:
import numpy as np
from parser import parse_model_from_txt

# Simplex

In [ ]:
class Simplex:
    """
        Implementa o algoritmo Simplex de duas fases para resolver problemas de programação linear.

        A classe é projetada para receber um modelo de PL já processado por um parser,
        transformar o problema para a forma padrão, aplicar a Fase 1 (para encontrar uma
        solução básica factível) e a Fase 2 (para encontrar a solução ótima), e então
        retornar a solução em termos das variáveis originais.

        **Args:**
            - *c_parser_vars* (**np.array**): Vetor de custos para as variáveis já transformadas (não-negativas) pelo parser.
            - *A_parser_vars* (**np.array**): Matriz de coeficientes das restrições para as variáveis transformadas.
            - *b_orig* (**np.array**): Vetor de termos independentes (lado direito) das restrições.
            - *signs* (**list**): Lista de strings com os sinais de cada restrição ('<=', '>=', '=').
            - *was_min* (**bool**): True se o problema original era de minimização.
            - *interpretation_info* (**dict**): Dicionário com informações para mapear a solução final de volta às variáveis originais.
            - *m* (**int**): Número de restrições.
            - *n_parser_vars* (**int**): Número de variáveis após o parser (antes de adicionar folga/artificiais).
            - *solution* (**dict**): Armazena o dicionário com o resultado final da otimização.
            - *status* (**str**): Armazena a string de status final da otimização.
            - *tableau* (**np.array**): O tableau do Simplex.
            - *basic_vars* (**list**): Lista de índices das variáveis que estão na base.
            - *num_vars* (**int**): Número total de variáveis no tableau (incluindo folga, excesso e artificiais).
            - *c_phase1* (**np.array**): Vetor de custos para o problema da Fase 1.
            - *artificial_indices* (**list**): Lista de índices das colunas correspondentes às variáveis artificiais no tableau.
    """
    def __init__(self, c_from_parser, A_from_parser, b, signs, was_min=False, interpretation_info=None):
        """
        Inicializa o solver Simplex.

        **Args:**
            - *c_from_parser* (**list**): Coeficientes da função objetivo para as variáveis pós-parser.
            - *A_from_parser* (**list[list]**): Matriz de coeficientes das restrições pós-parser.
            - *b* (**list**): Vetor de termos independentes (RHS).
            - *signs* (**list**): Sinais das restrições.
            - *was_min* (**bool, optional**): True se o problema original era de minimização. Defaults to False.
            - *interpretation_info* (**dict, optional**): Dados para reinterpretar a solução. Defaults to None.
        """
        # Estes são os coeficientes e matriz para variáveis já transformadas pelo parser (não-negativas)
        self.c_parser_vars = np.array(c_from_parser, dtype=float)
        self.A_parser_vars = np.array(A_from_parser, dtype=float) # Dimensões: m x n_parser_vars
        self.b_orig = np.array(b, dtype=float)                    # Vetor m x 1
        self.signs = signs
        self.was_min = was_min
        self.interpretation_info = interpretation_info
        self.m, self.n_parser_vars = self.A_parser_vars.shape

        self.solution = None
        self.status = None

        # Atributos para o Simplex Tabular (inicializados por _build_phase_one)
        self.tableau = None
        self.basic_vars = None # Para tabular: lista de índices de colunas no tableau
        self.num_vars = None   # Para tabular: número total de colunas de variáveis no tableau
        self.c_phase1 = None   # Para tabular: custos da fase 1
        self.artificial_indices = [] # Para tabular: índices das colunas artificiais no tableau


    def _build_phase_one(self):
        # Esta é a sua função _build_phase_one tabular.
        # Ela inicializa self.tableau, self.basic_vars, self.num_vars, 
        # self.c_phase1, self.artificial_indices.
        # Ela usa self.A_parser_vars e self.c_parser_vars como base.
        
        m, n_orig_transformed = self.A_parser_vars.shape
        
        # Garante que b seja não-negativo
        for i in range(m):
            if self.b_orig[i] < 0:
                self.b_orig[i] *= -1
                self.A_parser_vars[i, :] *= -1
                if self.signs[i] == '<=': self.signs[i] = '>='
                elif self.signs[i] == '>=': self.signs[i] = '<='
        
        num_slack_total = sum(1 for sign in self.signs if sign in ['<=', '>='])
        num_artificial_total = sum(1 for sign in self.signs if sign in ['>=', '='])
        
        
        self.num_vars = n_orig_transformed + num_slack_total + num_artificial_total # Num total de cols no tableau
        self.tableau = np.zeros((m, self.num_vars + 1))
        self.artificial_indices = [] # Índices no tableau
        current_basic_vars_tabular = [-1] * m

        slack_col_ptr = n_orig_transformed
        artificial_col_ptr = n_orig_transformed + num_slack_total

        for i in range(m):
            self.tableau[i, :n_orig_transformed] = self.A_parser_vars[i, :]
            self.tableau[i, -1] = self.b_orig[i]
            sign = self.signs[i]
            if sign == '<=':
                self.tableau[i, slack_col_ptr] = 1.0
                current_basic_vars_tabular[i] = slack_col_ptr
                slack_col_ptr += 1
            elif sign == '>=':
                self.tableau[i, slack_col_ptr] = -1.0 # Excesso
                slack_col_ptr += 1
                self.tableau[i, artificial_col_ptr] = 1.0 # Artificial
                self.artificial_indices.append(artificial_col_ptr)
                current_basic_vars_tabular[i] = artificial_col_ptr
                artificial_col_ptr += 1
            elif sign == '=':
                self.tableau[i, artificial_col_ptr] = 1.0 # Artificial
                self.artificial_indices.append(artificial_col_ptr)
                current_basic_vars_tabular[i] = artificial_col_ptr
                artificial_col_ptr += 1
        
        self.basic_vars = current_basic_vars_tabular
        # A função objetivo da fase 1 é minimizar a soma das variáveis artificiais,
        # o que é equivalente a maximizar -1 * (soma das artificiais).
        self.c_phase1 = np.zeros(self.num_vars)
        for art_idx in self.artificial_indices:
            self.c_phase1[art_idx] = 1.0

        if any(bv == -1 for bv in self.basic_vars):
             raise ValueError(f"Tabular _build_phase_one: Nem todas as variáveis básicas foram detectadas: {self.basic_vars}")



    def _simplex(self, c_tableau, phase):
        """
        Executa o algoritmo Simplex tabular para um dado tableau e vetor de custos.
        
        Args:
            c_tableau (np.array): Vetor de custos para a fase atual.
            phase (int): A fase atual (1 ou 2).

        Returns:
            dict: Um dicionário com o status, solução, valor e outras informações.
        """
        m, n_plus_1 = self.tableau.shape 
        current_tableau_num_vars = n_plus_1 - 1

        # Garante que o vetor de custos tenha o tamanho correto
        if len(c_tableau) != current_tableau_num_vars:
            temp_c = np.zeros(current_tableau_num_vars)
            temp_c[:len(c_tableau)] = c_tableau
            c_tableau = temp_c

        # Para a Fase 1, o tableau inicial pode não ser canônico para a F.O. artificial.
        # Precisamos zerar os custos reduzidos das variáveis artificiais que estão na base.
        if phase == 1:
            cb = c_tableau[self.basic_vars]
            zj = cb @ self.tableau[:, :-1]
            c_tableau = c_tableau - zj

        iteration = 0
        max_iterations = self.m * self.num_vars * 2 

        while iteration < max_iterations:
            # O cálculo do custo reduzido agora é unificado para ambas as fases
            cb = c_tableau[self.basic_vars]
            zj = cb @ self.tableau[:, :-1]
            cj_zj = c_tableau - zj

            # Critério de parada: todos os custos reduzidos são <= 0 (ótimo)
            if np.all(cj_zj <= 1e-9):
                solution_tableau = np.zeros(current_tableau_num_vars)
                for i, basic_var_idx in enumerate(self.basic_vars):
                    solution_tableau[basic_var_idx] = self.tableau[i, -1]
                
                # O valor da F.O. é sempre c . x
                current_obj_value = c_tableau @ solution_tableau

                # --- VERIFICAÇÃO: DEGENERESCÊNCIA E MÚLTIPLAS SOLUÇÕES ---
                is_degenerate = np.any(np.isclose(self.tableau[:, -1], 0))
                non_basic_indices = np.setdiff1d(np.arange(current_tableau_num_vars), self.basic_vars)
                has_multiple_solutions = np.any(np.isclose(cj_zj[non_basic_indices], 0))
                
                return {
                    'status': 'optimal', 'solution': solution_tableau, 'value': current_obj_value,
                    'is_degenerate': is_degenerate, 'has_multiple_solutions': has_multiple_solutions
                }

            # Escolha da variável que entra na base
            entering_col = np.argmax(cj_zj)
            
            # Critério de parada: solução ilimitada
            if np.all(self.tableau[:, entering_col] <= 1e-9):
                return {'status': 'unbounded', 'solution': None, 'value': np.inf}

            # Teste da razão para escolher a variável que sai da base
            ratios = np.full(self.m, np.inf)
            for i in range(self.m):
                if self.tableau[i, entering_col] > 1e-9:
                    ratios[i] = self.tableau[i, -1] / self.tableau[i, entering_col]
            
            leaving_row = np.argmin(ratios)
            pivot_element = self.tableau[leaving_row, entering_col]

            if abs(pivot_element) < 1e-9:
                return {'status': 'error_numerical_instability', 'solution': None, 'value': None}

            # Pivoteamento
            self.tableau[leaving_row, :] /= pivot_element
            for i in range(self.m):
                if i != leaving_row:
                    factor = self.tableau[i, entering_col]
                    self.tableau[i, :] -= factor * self.tableau[leaving_row, :]
            
            self.basic_vars[leaving_row] = entering_col
            iteration += 1
        
        return {'status': 'max_iterations_reached', 'solution': None, 'value': None}


    def solve(self, method='tabular'):
        """
        Executa o solver Simplex.
        """
        if method != 'tabular':
            raise ValueError("Método Simplex desconhecido. Escolha 'tabular'.")

        # --- Fase 1 ---
        self._build_phase_one()
        
        # Define o vetor de custo para a Fase 1: maximizar -w = -soma(x_a)
        c_phase1_objective = np.zeros(self.num_vars)
        if self.artificial_indices:
            c_phase1_objective[self.artificial_indices] = -1.0

        result_phase1 = self._simplex(c_phase1_objective, phase=1)
        
        # Verifica se a Fase 1 encontrou uma solução factível (valor da F.O. artificial == 0)
        if result_phase1.get('status') != 'optimal' or abs(result_phase1.get('value', float('inf'))) > 1e-6:
            status = 'infeasible' if result_phase1.get('status') == 'optimal' else result_phase1.get('status', 'infeasible_phase1_error')
            self.solution = {'status': status, 'solution': None, 'value': None}
            return self.solution

        # --- Fase 2 ---
        # Remover colunas artificiais para preparar para a Fase 2, se existirem
        if self.artificial_indices:
            vars_to_keep = [i for i in range(self.num_vars) if i not in self.artificial_indices]
            self.tableau = self.tableau[:, vars_to_keep + [self.num_vars]]
            
            map_old_to_new_idx = {old_idx: new_idx for new_idx, old_idx in enumerate(vars_to_keep)}
            self.basic_vars = [map_old_to_new_idx[bv] for bv in self.basic_vars]
            self.num_vars = len(vars_to_keep)

        self.c = np.zeros(self.num_vars)
        self.c[:len(self.c_parser_vars)] = self.c_parser_vars

        result_phase2 = self._simplex(self.c, phase=2)

        # --- Pós-processamento do resultado final ---
        final_result = {'status': result_phase2.get('status')}

        if final_result['status'] == 'optimal':
            # Reinterpreta a solução em termos das variáveis originais
            final_solution_orig_vars = {}
            solution_values = result_phase2['solution']
            for var_name, info in self.interpretation_info['simplex_vars_map'].items():
                cols = info['cols_parser']
                # Remapeia as colunas do parser para as colunas atuais da Fase 2
                # Esta parte é complexa se o parser e o simplex não estiverem perfeitamente alinhados
                # Assumindo que as colunas do parser são as primeiras `n_parser_vars`
                if info['type'] == 'free':
                    val_p = solution_values[cols[0]]
                    val_n = solution_values[cols[1]]
                    final_solution_orig_vars[var_name] = val_p - val_n
                else:
                    val = solution_values[cols[0]]
                    final_solution_orig_vars[var_name] = val * info['mult']

            ordered_solution_list = [final_solution_orig_vars[v] for v in self.interpretation_info['sorted_original_vars']]
            
            # O valor do objetivo da Fase 2 precisa ser recalculado com o c original
            # pois o _simplex o calcula com o c.
            original_c = np.array(self.interpretation_info['original_c_values']) if 'original_c_values' in self.interpretation_info else self.c_parser_vars
            final_value = original_c @ np.array(ordered_solution_list)

            final_result.update({
                'solution': ordered_solution_list,
                'value': -final_value if self.was_min else final_value,
                'is_degenerate': result_phase2.get('is_degenerate', False),
                'has_multiple_solutions': result_phase2.get('has_multiple_solutions', False)
            })
        else:
            final_result.update({
                'solution': None,
                'value': result_phase2.get('value'),
                'is_degenerate': False,
                'has_multiple_solutions': False
            })
            
        self.solution = final_result
        return self.solution

### Análise dos Parâmetros de Retorno da Função `parse_model_from_txt`

A função `parse_model_from_txt` retorna um único dicionário que encapsula todo o problema de Programação Linear (PL) em um formato estruturado, pronto para ser processado pelo algoritmo Simplex. Abaixo, detalhamos cada chave deste dicionário.

---

#### chave: `c`
* **Tipo**: `list` de `float`
* **Descrição**: Representa o vetor de custos, ou seja, os coeficientes da função objetivo.
* **Detalhes**:
    * Este vetor já está ajustado para o formato de maximização que o Simplex utiliza. Se o problema original for `min z`, a função o converte para `max -z`, e os coeficientes em `c` serão os do problema original multiplicados por -1.
    * Os coeficientes são mapeados para as variáveis do Simplex, que são todas não-negativas. Por exemplo, se a variável original `x1` for livre (transformada em `x1_p - x1_n`), um termo `5x1` no objetivo resultará em `5` e `-5` nas posições correspondentes de `c` para `x1_p` e `x1_n`.

---

#### chave: `A`
* **Tipo**: `list` de `list` de `float`
* **Descrição**: Representa a matriz `A` de coeficientes das restrições.
* **Detalhes**:
    * Cada lista interna corresponde a uma restrição (linha da matriz).
    * As colunas da matriz correspondem às variáveis do Simplex (ex: `x1_p`, `x1_n`, `x2`, ...) na ordem definida internamente pelo parser.

---

#### chave: `b`
* **Tipo**: `list` de `float`
* **Descrição**: Contém os termos independentes, ou seja, os valores do lado direito (RHS - Right-Hand Side) de cada restrição.
* **Detalhes**:
    * A ordem dos valores em `b` corresponde diretamente à ordem das restrições (linhas) na matriz `A`.

---

#### chave: `signs`
* **Tipo**: `list` de `str`
* **Descrição**: Armazena os sinais de relação (`<=`, `>=`, `=`) para cada uma das restrições.
* **Detalhes**:
    * A ordem dos sinais corresponde à ordem das restrições em `A` e `b`.

---

#### chave: `was_min`
* **Tipo**: `bool`
* **Descrição**: Um indicador booleano que "lembra" se o problema original era de minimização.
* **Detalhes**:
    * `True`: O problema original era de minimização.
    * `False`: O problema original era de maximização.
    * **Finalidade**: O solver Simplex é implementado para sempre maximizar. Esta flag é usada ao final do processo para determinar se o valor ótimo encontrado deve ser multiplicado por -1 para fornecer a resposta correta para o problema de minimização original.

---

#### chave: `interpretation_info`
* **Tipo**: `dict`
* **Descrição**: Este dicionário é fundamental. Ele contém todo o "mapa" necessário para traduzir a solução final, que está em termos de variáveis do Simplex, de volta para os termos das variáveis originais do problema.

##### Sub-chave: `sorted_original_vars`
* **Tipo**: `list` de `str`
* **Descrição**: Uma lista com os nomes das variáveis originais do problema (ex: `['x1', 'x2', 'x3']`), ordenadas para garantir uma apresentação de resultados consistente.

##### Sub-chave: `simplex_var_column_names`
* **Tipo**: `list` de `str`
* **Descrição**: Contém os nomes das variáveis que o Simplex efetivamente utiliza. Por exemplo, se `x1` for uma variável livre, esta lista poderia conter `['x1_p', 'x1_n', 'x2']`.

##### Sub-chave: `simplex_vars_map`
* **Tipo**: `dict`
* **Descrição**: O núcleo da tradução. É um dicionário que mapeia o nome de cada variável **original** para um outro dicionário que detalha sua transformação.
    * **`type` (`str`)**: O tipo da variável original. Pode ser:
        * `'non_negative'`: Para variáveis $x_i \ge 0$.
        * `'free'`: Para variáveis livres.
        * `'negative'`: Para variáveis $x_i \le 0$.
    * **`cols_parser` (`list[int]`)**: Os índices das colunas que a variável original ocupa na matriz `A` e no vetor `c`. Uma variável não-negativa terá um único índice (ex: `[0]`), enquanto uma livre terá dois (ex: `[0, 1]`).
    * **`mult` (`int`)**: Um multiplicador usado na transformação dos coeficientes. É `1` para variáveis não-negativas/livres e `-1` para variáveis negativas (na transformação `x_orig = -x_prime`).
    * **`simplex_names` (`list[str]`)**: Os nomes formais dados às variáveis do Simplex que representam a variável original (ex: `['x1_p', 'x1_n']`).

# Converter Modelo no TXT

In [32]:
model = parse_model_from_txt(r'./modelos/modelo_online1.txt')
print(model)


{'c': [1.0, 2.0, -1.0], 'A': [[2.0, 1.0, 1.0], [4.0, 2.0, 3.0], [2.0, 5.0, 5.0]], 'b': [14.0, 28.0, 30.0], 'signs': ['<=', '<=', '<='], 'was_min': False, 'interpretation_info': {'sorted_original_vars': ['x1', 'x2', 'x3'], 'simplex_vars_map': {'x1': {'type': 'non_negative', 'cols_parser': [0], 'mult': 1, 'simplex_names': ['x1']}, 'x2': {'type': 'non_negative', 'cols_parser': [1], 'mult': 1, 'simplex_names': ['x2']}, 'x3': {'type': 'non_negative', 'cols_parser': [2], 'mult': 1, 'simplex_names': ['x3']}}, 'simplex_var_column_names': ['x1', 'x2', 'x3']}}


In [33]:
simplex = Simplex(model['c'], model['A'], 
                  model['b'], model['signs'], was_min=model['was_min'], 
                  interpretation_info=model['interpretation_info'])

result = simplex.solve()
print(result)

{'status': 'optimal', 'solution': [5.0, 4.0, 0.0], 'value': 13.0, 'is_degenerate': True, 'has_multiple_solutions': False}
